In [1]:
import fn_module

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = fn_module.read_file("CSE508_Winter2023_Dataset_Processed_new")
print("File count = ",len(data))

File count =  1400


In [3]:
keys = list(data.keys())

In [4]:
s = ""
df = []
for key in data:
    df.append(data[key])


In [5]:
from collections import defaultdict

def create_inverted_index(documents):
    inverted_index = defaultdict(list)
    
    for doc_id, document in enumerate(documents):
        for word in document.split():
            inverted_index[word].append(doc_id)
    
    return inverted_index

In [6]:
inverted_index = create_inverted_index(df)

In [7]:

import pickle
from collections import defaultdict
def save_inverted_index(inverted_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(inverted_index, f)

def load_inverted_index(file_path):
    with open(file_path, 'rb') as f:
        inverted_index = pickle.load(f)
    
    return inverted_index

In [8]:
file_path = 'inverted_index.pickle'
save_inverted_index(inverted_index, file_path)

inverted_index = load_inverted_index(file_path)

In [73]:
def get_list(inverted_index , word):
    if word in inverted_index:
        return inverted_index[word]
    else:
        return []
    # return inverted_index[word]

In [10]:
def and_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
    return final_set, count_comparisons

In [11]:
def or_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
        
    while(ptr_2 < len_2):
        final_set.add(s_2[ptr_2])
        ptr_2 += 1
        
    result = sorted(final_set)
        
    return result, count_comparisons

In [97]:
def ornot_operation(s1, s2):
    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            ptr_2 = ptr_2 + 1
        else:
            final_set.add(s_1[ptr_1])
            ptr_1 = ptr_1 + 1
        count_comparisons += 1
        
    while(ptr_1 < len_1):
        final_set.add(s_1[ptr_1])
        ptr_1 += 1
    
    s1 = set(s1)
    s2 = set(s2)
    temp = []
    for i in range(1400):
        temp.append(i)
    temp = set(temp)
    not_s2 = temp - s2
    res = list(sorted(s1.union(not_s2)))
    res,cnt = or_operation(s1,not_s2)
    res = sorted(res)
    result = sorted(final_set)
        
    return res, cnt

In [79]:
def andnot_operation(s1, s2):

    ptr_1 = 0
    ptr_2 = 0
    final_set = set()
    s_1 = sorted(s1)
    s_2 = sorted(s2)
    count_comparisons = 0
    len_1 = len(s_1)
    len_2 = len(s_2)
    while(ptr_1 < len_1 and ptr_2 < len_2):
        if s_1[ptr_1]==s_2[ptr_2]:
            ptr_1 = ptr_1 + 1
            ptr_2 = ptr_2 + 1
        elif s_1[ptr_1] > s_2[ptr_2]:
            # final_set.add(s_2[ptr_2])
            ptr_2 = ptr_2 + 1
        else:
            ptr_1 = ptr_1 + 1
        count_comparisons += 1

    s1 = set(s1)
    s2 = set(s2)
    # print("/"*100)
    # print("S1 set =",s1)
    # print("S2 set =",s2)
    # print("/"*100)

    final_set = s1 - s2
    return final_set, count_comparisons

In [106]:
def retrieve_documents(output_sequence, operations , inverted_index, log = False):
    count = 0
    l1 = get_list(inverted_index,output_sequence[0])
    if log:
        print("L1 = ",sorted(list(set(l1))))
    for i in range(len(operations)):
        count_temp = 0
        l2 = get_list(inverted_index,output_sequence[i+1])
        if log:
            print("L2 = ",sorted(list(set(l2))))
        op = []
        temp = operations[i].split()
        if(len(temp)==2):
            operations[i]=temp[0].lower()+temp[1].lower()
        if(operations[i].lower() == 'andnot'):
            print("IN AND NOT FUN CALL","%"*100)
            op, count_temp = andnot_operation(l1,l2)
        elif(operations[i].lower() == 'and'):
            op, count_temp = and_operation(l1,l2)
        elif(operations[i].lower() == 'or'):
            op, count_temp = or_operation(l1,l2)
        elif(operations[i] == 'ornot'):
            op, count_temp = ornot_operation(l1,l2)
        if log:
            print("OPERATOR = ",operations[i])
            print("OUTPUT AFTER OPERTION",sorted(list(set(op))))
            print("-"*100)
        
        l1 = op
        count+=count_temp

    return l1, count

In [107]:
t = int(input())

for _ in range(t):
    sequence  = input()
    oper = input()

    output_sequence = fn_module.preprocess(sequence)
 
    operations = oper.split(',')

    print("Processed sequence :",output_sequence)
    
    query = output_sequence[0]+' '
    i = 0
    while( i < len(operations)):
        query = query + operations[i] + ' ' + output_sequence[i+1] + ' '
        i= i+1
    
    print("Final Query :",query)

    num_documents, operation_count = retrieve_documents(output_sequence, operations,inverted_index,True)
    # num_documents, operation_count = retrieve_documents(output_sequence, operations,inverted_index)
    
    print("Number of documents retrieved for query "+str(_+1)+" is :",len(num_documents))
    print("Names of the documents retrieved for "+str(_+1)+" is :")
    
    if len(num_documents)==0:
        print("NO DOCUMENT FOUND!!")
    else:
        doc_lst = []
        for val in num_documents:
            doc_lst.append(keys[val])
        print(doc_lst)

    print("Number of comparisons required for query "+str(_+1)+" is :",operation_count)
    print("-"*150)

    
        

Processed sequence : ['bluntnosed', 'slender', 'larger']
Final Query : bluntnosed or slender  or not larger 
L1 =  [24, 100, 138, 159, 293, 348, 353, 369, 422, 455, 472, 543, 575, 592, 596, 975, 999, 1005, 1081, 1252, 1280, 1355, 1392]
L2 =  [24, 26, 43, 59, 62, 68, 122, 145, 146, 159, 190, 191, 204, 216, 228, 229, 234, 247, 249, 258, 272, 288, 306, 325, 331, 353, 358, 431, 432, 455, 463, 464, 467, 468, 492, 493, 512, 519, 527, 540, 544, 555, 556, 568, 571, 576, 598, 599, 600, 604, 624, 682, 687, 738, 745, 751, 790, 791, 800, 801, 900, 901, 917, 920, 921, 924, 938, 940, 968, 1111, 1185, 1196, 1200, 1209, 1212, 1247, 1258, 1299, 1304, 1309, 1327, 1372, 1380, 1392]
OPERATOR =  or
OUTPUT AFTER OPERTION [24, 26, 43, 59, 62, 68, 100, 122, 138, 145, 146, 159, 190, 191, 204, 216, 228, 229, 234, 247, 249, 258, 272, 288, 293, 306, 325, 331, 348, 353, 358, 369, 422, 431, 432, 455, 463, 464, 467, 468, 472, 492, 493, 512, 519, 527, 540, 543, 544, 555, 556, 568, 571, 575, 576, 592, 596, 598, 599, 6

In [51]:
print(inverted_index["canister"])

[]


In [17]:
len(inverted_index.keys())

8969